# Feed Forward neural network

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
torch.cuda.is_available()

False

In [4]:
# Hyper parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [6]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data',
                                          train=True,
                                          transform=transforms.ToTensor(),
                                          download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data',
                                           train=False,
                                           transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)



Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw




Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(
                epoch+1, num_epochs, i+1, total_step, loss.item()
            ))

Epoch [1/5], Step [100/600], Loss: 0.4132
Epoch [1/5], Step [200/600], Loss: 0.3877
Epoch [1/5], Step [300/600], Loss: 0.3535
Epoch [1/5], Step [400/600], Loss: 0.1386
Epoch [1/5], Step [500/600], Loss: 0.1380
Epoch [1/5], Step [600/600], Loss: 0.3158
Epoch [2/5], Step [100/600], Loss: 0.0960
Epoch [2/5], Step [200/600], Loss: 0.0767
Epoch [2/5], Step [300/600], Loss: 0.1802
Epoch [2/5], Step [400/600], Loss: 0.0372
Epoch [2/5], Step [500/600], Loss: 0.1271
Epoch [2/5], Step [600/600], Loss: 0.1238
Epoch [3/5], Step [100/600], Loss: 0.0446
Epoch [3/5], Step [200/600], Loss: 0.0991
Epoch [3/5], Step [300/600], Loss: 0.0446
Epoch [3/5], Step [400/600], Loss: 0.0269
Epoch [3/5], Step [500/600], Loss: 0.1224
Epoch [3/5], Step [600/600], Loss: 0.0228
Epoch [4/5], Step [100/600], Loss: 0.0370
Epoch [4/5], Step [200/600], Loss: 0.0282
Epoch [4/5], Step [300/600], Loss: 0.0383
Epoch [4/5], Step [400/600], Loss: 0.0129
Epoch [4/5], Step [500/600], Loss: 0.0191
Epoch [4/5], Step [600/600], Loss:

In [11]:
# Test the model
# don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.99 %


In [12]:
# save the model checkpoint
torch.save(model.state_dict(), "model.ckpt")